In [115]:
# scikit-learn                      0.23.2
# sklearn-crfsuite                  0.3.6
!pip install scikit-learn==0.23.2
!pip install sklearn-crfsuite==0.3.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [116]:
!pip install bnlp_toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [117]:
!git clone https://github.com/sagorbrur/bnlp/ --depth 1

fatal: destination path 'bnlp' already exists and is not an empty directory.


In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
# from bnlp import POS
# bn_pos = POS()
# model_path = "bnlp/model/bn_pos.pkl"
# text = ["আমি", 'ভাত', 'খাই'] # or you can pass ['আমি', 'ভাত', 'খাই', '।']
# for t in text:
#   res = bn_pos.tag(model_path, t)
#   print(res)


In [120]:
# dir(POS)

In [121]:
from bnlp.corpus import stopwords, punctuations, letters, digits

# print(stopwords)
# print(punctuations)
# print(letters)
# print(digits)

In [122]:
# from bnlp import NER
# bn_ner = NER()
# model_path = "bnlp/model/bn_ner.pkl"
# text = "সে ঢাকায় থাকে।" # or you can pass ['সে', 'ঢাকায়', 'থাকে', '।']
# result = bn_ner.tag(model_path, text)
# print(result)
# # [('সে', 'O'), ('ঢাকায়', 'S-LOC'), ('থাকে', 'O')]

In [123]:
# from bnlp import POS
# bn_pos = POS()
# model_path = "bnlp/model/bn_pos.pkl"
# text = ["ঢাকা ", 'হতে', 'দূরে'] # or you can pass ['আমি', 'ভাত', 'খাই', '।']
# for t in text:
#   res = bn_pos.tag(model_path, t)
#   print(res)


In [124]:

from bnlp import POS
bn_pos = POS()
model_path = "bnlp/model/bn_pos.pkl"

def isPPR(text): 
  if(bn_pos.tag(model_path, text) == "PPR"): 
    # print("Yes PPR")
    return True
  return False 

def isStopWord(text):
  global stopwords
  return text in stopwords

def isDigit(text):
  global digits
  return text in digits

def isPuncutuation(text): 
  global punctuations
  return text in punctuations

# from bnlp import POS
# bn_pos = POS()
# model_path = "model/bn_pos.pkl"
# text = "আমি ভাত খাই।" # or you can pass ['আমি', 'ভাত', 'খাই', '।']
# res = bn_pos.tag(model_path, text)
# print(res)
# [('আমি', 'PPR'), ('ভাত', 'NC'), ('খাই', 'VM'), ('।', 'PU')]
def getBNPos(sentence):
  # Focus: uporer gulate text = word; ekhane kintu pura sentence pass kortesi 
  return bn_pos.tag(model_path, sentence)



In [125]:
out1 = getBNPos("সে ঢাকায় থাকে।")
print(type(out1))
print(len(out1))
print(out1)

<class 'list'>
4
[('সে', 'PPR'), ('ঢাকায়', 'VM'), ('থাকে', 'VAUX'), ('।', 'PU')]


In [126]:
# from bnlp import POS
# bn_pos = POS()
# model_path = "bnlp/model/bn_pos.pkl"
# text = "ঢাকা হতে দূরে ।" # or you can pass ['আমি', 'ভাত', 'খাই', '।']
# res = bn_pos.tag(model_path, text)
# print(res)


In [127]:
# ! pip install -U 'scikit-learn<0.24'

In [128]:
# !pip install sklearn_crfsuite

In [129]:
!rm dev_generic_no_punc.csv
!rm train_generic_no_punc.csv

In [130]:
!cp "/content/drive/MyDrive/Colab Notebooks/NLP_Hackathon/dev_generic_no_punc.csv" dev_generic_no_punc.csv
!cp "/content/drive/MyDrive/Colab Notebooks/NLP_Hackathon/train_generic_no_punc.csv" train_generic_no_punc.csv

In [131]:
import pandas as pd

from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics

In [132]:
def extract_sentence_features(df):
    # print("df:", df)
    sentence_length = len(df)
    sentence1 = ""
  
    for i in range(sentence_length):
        sentence1 += df.iloc[i]['word']
        # if i<n-1:
        sentence1 += " "

    postag1 = getBNPos(sentence1)
    # print(sentence1)
    # print(postag1)


    for i in range(sentence_length):
        token = df.iloc[i]['word']
        # print("i:", i)
        
        
        
        features = {
            'bias': 1.0,
            'token.lower()': token.lower(),
            # 'isPPR(token)': isPPR(token),
            'isStopWord(token)': isStopWord(token),
            'token.isupper()': token.isupper(),
            'token.istitle()': token.istitle(),
            # 'token.isdigit()': token.isdigit()
            'isDigit(token)': isDigit(token),
            'isPuncutuation(token)': isPuncutuation(token),
        }
        # print("i:", i, " len(postag1):", len(postag1))
        features['postag'] = postag1[i][1]
        if i > 0:
            previous_token = df.iloc[i-1]['word']
            features.update({
                'previous_token.lower()': previous_token.lower(),
                # 'isPPR(previous_token)': isPPR(previous_token),
                'isStopWord(previous_token)': isStopWord(previous_token),
                'previous_token.isupper()': previous_token.isupper(),
                'previous_token.istitle()': previous_token.istitle(),
                # 'previous_token.isdigit()': previous_token.isdigit()
                'isDigit(previous_token)': isDigit(previous_token),
                'isPuncutuation(previous_token)': isPuncutuation(previous_token),
            })
        else:
            features['BOS'] = True

        if i < sentence_length - 1:
            posterior_token = df.iloc[i+1]['word']
            features.update({
                'posterior_token.lower()': posterior_token.lower(),
                # 'isPPR(posterior_token)': isPPR(posterior_token),
                'isStopWord(posterior_token)': isStopWord(posterior_token),
                'posterior_token.isupper()': posterior_token.isupper(),
                'posterior_token.istitle()': posterior_token.istitle(),
                # 'posterior_token.isdigit()': posterior_token.isdigit()
                'isDigit(posterior_token)': isDigit(posterior_token),
                'isPuncutuation(posterior_token)': isPuncutuation(posterior_token),
            })
        else:
            features['EOS'] = True

        yield features

In [133]:
from tqdm import tqdm
def prepare(df, include_y=False):
    X, y = [], []
    for _, group_df in tqdm(df.groupby(['sentence_id'])):
        X.append(list(extract_sentence_features(group_df)))
        if include_y:
            y.append(group_df['tag'])
    if include_y:
        return X, y
    return X

In [134]:
train_df = pd.read_csv('train_generic_no_punc.csv')
validation_df = pd.read_csv('dev_generic_no_punc.csv')


In [135]:
train_df.head()

,sentence_id,word,tag
0,0,তার,O
1,0,মৃত্যুর,O
2,0,দশ,O
3,0,দিন,O
4,0,পর,O


In [136]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191897 entries, 0 to 191896
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   sentence_id  191897 non-null  int64 
 1   word         191897 non-null  object
 2   tag          191897 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.4+ MB


In [137]:
validation_df.head()

,sentence_id,word,tag
0,0,তিনি,O
1,0,যুবক,O
2,0,হিসেবে,O
3,0,শেফিল্ড,B-GRP
4,0,বুধবার,I-GRP


In [138]:
validation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10333 entries, 0 to 10332
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sentence_id  10333 non-null  int64 
 1   word         10333 non-null  object
 2   tag          10333 non-null  object
dtypes: int64(1), object(2)
memory usage: 242.3+ KB


In [139]:
train_df['word'].fillna('', inplace=True)
validation_df['word'].fillna('', inplace=True)


In [140]:
X_train, y_train = prepare(train_df, include_y=True)
X_validation, y_validation = prepare(validation_df, include_y=True)
# X_train, y_train = prepare(train_df[0:1000], include_y=True)
# X_validation, y_validation = prepare(validation_df[0:1000], include_y=True)


100%|██████████| 800/800 [00:19<00:00, 41.52it/s]


In [141]:
X_train[0]

[{'bias': 1.0,
  'token.lower()': 'তার',
  'isStopWord(token)': True,
  'token.isupper()': False,
  'token.istitle()': False,
  'isDigit(token)': False,
  'isPuncutuation(token)': False,
  'postag': 'PPR',
  'BOS': True,
  'posterior_token.lower()': 'মৃত্যুর',
  'isStopWord(posterior_token)': False,
  'posterior_token.isupper()': False,
  'posterior_token.istitle()': False,
  'isDigit(posterior_token)': False,
  'isPuncutuation(posterior_token)': False},
 {'bias': 1.0,
  'token.lower()': 'মৃত্যুর',
  'isStopWord(token)': False,
  'token.isupper()': False,
  'token.istitle()': False,
  'isDigit(token)': False,
  'isPuncutuation(token)': False,
  'postag': 'NC',
  'previous_token.lower()': 'তার',
  'isStopWord(previous_token)': True,
  'previous_token.isupper()': False,
  'previous_token.istitle()': False,
  'isDigit(previous_token)': False,
  'isPuncutuation(previous_token)': False,
  'posterior_token.lower()': 'দশ',
  'isStopWord(posterior_token)': False,
  'posterior_token.isupper()':

In [142]:
len(X_train)

15300

In [143]:
len(X_validation)

800

In [144]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=500,
    all_possible_transitions=True,
    # verbose=True,
)

In [145]:
X_train[0]

[{'bias': 1.0,
  'token.lower()': 'তার',
  'isStopWord(token)': True,
  'token.isupper()': False,
  'token.istitle()': False,
  'isDigit(token)': False,
  'isPuncutuation(token)': False,
  'postag': 'PPR',
  'BOS': True,
  'posterior_token.lower()': 'মৃত্যুর',
  'isStopWord(posterior_token)': False,
  'posterior_token.isupper()': False,
  'posterior_token.istitle()': False,
  'isDigit(posterior_token)': False,
  'isPuncutuation(posterior_token)': False},
 {'bias': 1.0,
  'token.lower()': 'মৃত্যুর',
  'isStopWord(token)': False,
  'token.isupper()': False,
  'token.istitle()': False,
  'isDigit(token)': False,
  'isPuncutuation(token)': False,
  'postag': 'NC',
  'previous_token.lower()': 'তার',
  'isStopWord(previous_token)': True,
  'previous_token.isupper()': False,
  'previous_token.istitle()': False,
  'isDigit(previous_token)': False,
  'isPuncutuation(previous_token)': False,
  'posterior_token.lower()': 'দশ',
  'isStopWord(posterior_token)': False,
  'posterior_token.isupper()':

In [146]:
crf.fit(X_train, y_train)
# try:
#     crf.fit(X_train, y_train)
# except AttributeError:
#     pass

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=500)

In [147]:
y_pred = crf.predict(X_validation)

In [148]:
y_validation[0]

0         O
1         O
2         O
3     B-GRP
4     I-GRP
5     I-GRP
6         O
7         O
8         O
9         O
10        O
11        O
12        O
13        O
14        O
15        O
16        O
Name: tag, dtype: object

In [149]:
y_pred[0]

['O',
 'O',
 'O',
 'B-GRP',
 'I-GRP',
 'I-GRP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [150]:
print(metrics.flat_classification_report(y_validation, y_pred, digits=4))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


              precision    recall  f1-score   support

      B-CORP     0.8161    0.5591    0.6636       127
        B-CW     0.7455    0.3417    0.4686       120
       B-GRP     0.7921    0.6780    0.7306       118
       B-LOC     0.7191    0.6337    0.6737       101
       B-PER     0.7708    0.7708    0.7708       144
      B-PROD     0.7687    0.5421    0.6358       190
      I-CORP     0.8241    0.7295    0.7739       122
        I-CW     0.8101    0.3975    0.5333       161
       I-GRP     0.8318    0.7876    0.8091       226
       I-LOC     0.8444    0.6230    0.7170        61
       I-PER     0.7872    0.8222    0.8043       180
      I-PROD     0.7708    0.5736    0.6578       129
           O     0.9521    0.9894    0.9704      8654

    accuracy                         0.9313     10333
   macro avg     0.8025    0.6499    0.7084     10333
weighted avg     0.9259    0.9313    0.9252     10333



In [151]:
# crf

In [152]:
print("Done")

Done


In [162]:
import joblib

# save the model to a file
joblib.dump(crf, 'crf_model_POS_added_no_punc_500_epoch.joblib')

# later, to load the model from the file
loaded_crf = joblib.load('crf_model_POS_added_no_punc_500_epoch.joblib')


In [154]:
# !cp crf_model.joblib "/content/drive/MyDrive/Colab Notebooks/NLP_Hackathon/"

In [155]:
# print("RandomForest")

In [156]:
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators = 100)


In [157]:
# len(X_train)


In [158]:
# X_train[0]

In [159]:
# clf.fit(X_train, y_train)

In [160]:
# y_pred = clf.predict(X_validation)

In [161]:
# print(metrics.flat_classification_report(y_validation, y_pred, digits=4))